5. When a decision tree for regression performes worse than linear regression?  

Decision trees work better when the dataset is non-linear, and they handle collinearity better. They do a better work with categorical variables, with a mix of categorical and numerical features and with missing features

# Задача

Develop a model for predicting review rating.
Multiclass classification into 5 classes
Score: F1 with macro averaging
You are forbidden to use test dataset for any kind of training.
Remember proper training pipeline.
If you are not using default params in the models, you have to use some validation scheme to justify them.

Use random_state or seed params - your experiment must be reprodusible.

In [0]:
import pandas as pd
import numpy as np

SEED=1337
np.random.seed(SEED)

In [6]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.shape

(48192, 3)

In [15]:
df_test.shape

(5355, 3)

In [7]:
df_train.head()

,review,title,target
0,"The staff was very friendly, the breakfast ver...",Walker Gem,5
1,Excellent service - very approachable and prof...,Excellent Service,4
2,Really a top notch place to spend a day at the...,"Good location, warm and friendly staff",5
3,"a little noisy, there was a false fire alarm a...","nice hotel,",4
4,Place had too many animals and I'm allergic to...,Experience,3


In [8]:
df_test.head()

,review,title,target
0,"I am from old town, and I stayed in this hotel...",Incredible Hotel,5
1,We have been coming to the Ocean Park Inn for ...,We Love this beach front Inn,5
2,Perfect place for a quick get away. We had a q...,Love this place!,5
3,"The room was not the best however, it was good...",Good For One Night Stay...,2
4,Sous le motif d'une priode hivernale (inaccept...,Moyen,3


In [51]:
import nltk 
from nltk import ngrams
from nltk.text import Text 
nltk.download("stopwords") 
#--------# 
from nltk.corpus import stopwords 
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer

#Create lemmatizer and stopwords list 
english_stopwords = stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
import re

In [0]:
tokenizer = TreebankWordTokenizer()

In [0]:
data = df_train

In [19]:
data.append(df_test)

,review,title,target
0,"The staff was very friendly, the breakfast ver...",Walker Gem,5
1,Excellent service - very approachable and prof...,Excellent Service,4
2,Really a top notch place to spend a day at the...,"Good location, warm and friendly staff",5
3,"a little noisy, there was a false fire alarm a...","nice hotel,",4
4,Place had too many animals and I'm allergic to...,Experience,3
...,...,...,...
5350,We were in Houston for a family event so sever...,"Luxurious, comfortable hotel & spa",5
5351,"I stay in a lot of hotels, especially for work...",One of the best hotel rooms I've stayed in,5
5352,I really enjoyed my stay at The River Hotel! I...,Great Stay!,4
5353,Three of us stayed in the hotel during the rec...,A great hotel!!,5


In [48]:
clean_review = []
for i in data['review']:
  j = re.sub(r'[^\w\s]','', i)
  clean_review.append(j)
data['review word number'] = [len(tokenizer.tokenize(i)) for i in clean_review]
data.head()

,review,title,target,review word number
0,"The staff was very friendly, the breakfast ver...",Walker Gem,5,12
1,Excellent service - very approachable and prof...,Excellent Service,4,38
2,Really a top notch place to spend a day at the...,"Good location, warm and friendly staff",5,60
3,"a little noisy, there was a false fire alarm a...","nice hotel,",4,14
4,Place had too many animals and I'm allergic to...,Experience,3,45


In [0]:
data = data.drop(columns=['word number'])

In [0]:
lemmatizer = WordNetLemmatizer()

lemm_review = []
for i in clean_review:
  arr = tokenizer.tokenize(i) 
  new_arr = []
  text = ''
  for j in arr: 
    word = j.lower()
    if word not in english_stopwords:
      new_arr.append(lemmatizer.lemmatize(word))
  text = ' '.join(new_arr)
  lemm_review.append(text)
data['lemmas review'] = [i for i in lemm_review]

In [0]:
data.head()

In [80]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

from gensim.test.utils import get_tmpfile

fname = get_tmpfile("my_doc2vec_model")

model.save(fname)
model = Doc2Vec.load(fname)  # you can continue training with the loaded model!

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [0]:
data['vec_review'] = [np.array(model.infer_vector(i.split())) for i in data['lemmas review']]

In [0]:
data['vec_title'] = [np.array(model.infer_vector(i.lower().split())) for i in data['title']]

In [115]:
data.head()

,review,title,target,review word number,lemmas review,vec_review,vec_title
0,"The staff was very friendly, the breakfast ver...",Walker Gem,5,12,staff friendly breakfast nice extremely comfor...,"[0.010194686, 0.016370963, 0.093328126, -0.091...","[0.04870353, -0.075052395, -0.027693031, -0.02..."
1,Excellent service - very approachable and prof...,Excellent Service,4,38,excellent service approachable professional st...,"[0.061459873, 0.01002281, -0.01876115, -0.0216...","[0.041421738, 0.04850113, 0.0860733, 0.0072168..."
2,Really a top notch place to spend a day at the...,"Good location, warm and friendly staff",5,60,really top notch place spend day beginning end...,"[0.041237194, -0.013673915, -0.0091772005, -0....","[-0.029623618, 0.04343617, -0.07957134, -0.029..."
3,"a little noisy, there was a false fire alarm a...","nice hotel,",4,14,little noisy false fire alarm midnight reason ...,"[-0.05035006, 0.023602638, -0.09586342, 0.0674...","[-0.0021660826, -0.011124977, 0.005384014, -0...."
4,Place had too many animals and I'm allergic to...,Experience,3,45,place many animal im allergic petsalthough rec...,"[0.0961574, -0.012306325, 0.029194854, -0.0510...","[0.05104129, -0.061222367, 0.07374523, -0.0076..."


In [116]:
data_train = data[:48192]
data_train.shape

(48192, 7)

In [117]:
data_test = data[:5355]
data_test.shape

(5355, 7)

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import scipy.sparse as sp

In [0]:
from sklearn import metrics

In [124]:
X_train = data_train[['review word number']]
y_train = data_train['target']

param = [{'n_estimators': [15, 30, 45, 60]}]

#rf_classifier = RandomForestClassifier(random_state=SEED)
lr_model = LogisticRegressionCV(Cs=5, cv=5, n_jobs=-1, scoring='f1_macro', penalty='l2', solver='newton-cg', multi_class='multinomial', random_state=SEED)
#grid_search = GridSearchCV(rf_classifier, param, n_jobs=-1, cv=5, 
                           #scoring='f1_macro', verbose=1)
#grid_search.fit(X_train, y_train)

lr_model.fit(X_train, y_train)

LogisticRegressionCV(Cs=5, class_weight=None, cv=5, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='multinomial', n_jobs=-1,
                     penalty='l2', random_state=1337, refit=True,
                     scoring='f1_macro', solver='newton-cg', tol=0.0001,
                     verbose=0)

In [0]:
target_pred = lr_model.predict(data_test[['review word number']])

In [108]:
metrics.f1_score(target_pred, data_test['target'], average='macro')

0.12040912200075833

In [137]:
# НУ ПОЧЕМУ С ВЕКТОРАМИ НЕ РАБОТАЕТ!!!!!!...............

for_rev = np.vstack(data_train['vec_review'])
for_title = np.vstack(data_train['vec_title'])
X_train = np.hstack([for_rev, for_title])
y_train = data_train['target']

param = [{'n_estimators': [15, 30, 45, 60]}]

#rf_classifier = RandomForestClassifier(random_state=SEED)
lr_model = LogisticRegressionCV(Cs=5, cv=5, n_jobs=-1, scoring='f1_macro', penalty='l2', solver='newton-cg', multi_class='multinomial', random_state=SEED)
#grid_search = GridSearchCV(rf_classifier, param, n_jobs=-1, cv=5, 
                           #scoring='f1_macro', verbose=1)
#grid_search.fit(X_train, y_train)

lr_model.fit(X_train, y_train)

LogisticRegressionCV(Cs=5, class_weight=None, cv=5, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='multinomial', n_jobs=-1,
                     penalty='l2', random_state=1337, refit=True,
                     scoring='f1_macro', solver='newton-cg', tol=0.0001,
                     verbose=0)

In [0]:
for_rev_test = np.vstack(data_test['vec_review'])
for_title_test = np.vstack(data_test['vec_title'])

target_pred = lr_model.predict(np.hstack([for_rev_test, for_title_test]))

In [143]:
metrics.f1_score(target_pred, data_test['target'], average='macro')

0.11617861401881543